In [ ]:
import pandas as pd
import numpy as np
import json
import requests
import codecs
import pathlib

In [ ]:
# pip install python-certifi-win32

In [ ]:
from utilities.load_tables import (
    load_cadastro,
    _load_table
)

In [ ]:
def GET_API(link):
    # Set the request parameters
    url = link
    user = 'xxxx@xxxxxxxxx.com.br/token'
    pwd = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxx'

    # Do the HTTP get request
    response = requests.get(url, auth=(user, pwd),verify = False)

    # Check for HTTP codes other than 200
    if response.status_code != 200:
        print('Status:', response.status_code, 'Problem with the request. Exiting.')
        exit()

    # Decode the JSON response into a dictionary and use the data
    data = response.json()
    return data

In [ ]:
df_cadastro1 = load_cadastro()

df_cadastro1

In [ ]:
df_cadastro = df_cadastro1[['cod_loja','email_exec', 'celula']].drop_duplicates()

df_cadastro

In [ ]:
n = 1
i = 1
url_users = 'https://xxxxxxxxxx.zendesk.com/api/v2/users'
while i <2 :
    print(f'Página {n}: {url_users}')
    
    lista_users = GET_API(url_users)
    
    if n == 1:
        df_users1 = pd.DataFrame(lista_users['users'])
    else:
        df_aux = pd.DataFrame(lista_users['users'])
        df_users1 = pd.concat([df_users1, df_aux])
    
    prox_pagina = lista_users['next_page']
    if prox_pagina is None:
        prox_pagina_url = url_users.partition('=')[0] + "=" + str(n+1)
        lista_prox_pagina = GET_API(prox_pagina_url)
        if len(lista_prox_pagina['users']):
            url_users = prox_pagina_url
            n = n+1
        else:
            i = i+1
    else:
        url_users = lista_users['next_page']
        n = n+1
    
df_users1 = df_users1.reset_index()

In [ ]:
df_users= df_users1[['email', 'id']]

df_users

In [ ]:
lake_mail = 'xxxxxxxxxxxxx'

table_mail = 'xxxxxxxxxxxxx'

columns_mail = 'cod, loja, xmail,d_e_l_e_t_'

df_mail1 = _load_table(lake_mail, table_mail, columns_mail)

df_mail = df_mail1.loc[df_mail1['d_e_l_e_t_']!='*']

df_mail['cod_loja'] = df_mail['cod'] + "-" + df_mail['loja']

df_mail = df_mail [['cod', 'xmail']]

df_mail

In [ ]:
n = 1
i = 1
url_groups = 'https://xxxxxxxxxxxxx.zendesk.com/api/v2/groups'
while i <2 :
    print(f'Página {n}: {url_groups}')
    
    lista_groups = GET_API(url_groups)
    
    if n == 1:
        df_groups1 = pd.DataFrame(lista_groups['groups'])
    else:
        df_aux = pd.DataFrame(lista_groups['groups'])
        df_groups1 = pd.concat([df_groups1, df_aux])
    
    url_groups = lista_groups['next_page']
    if url_groups is None:
        i = i+1
    else:
        n = n+1
    
df_groups1 = df_groups1.reset_index()

df_groups1

In [ ]:
df_groups = df_groups1[['id', 'name']]

df_groups = df_groups.rename( columns = {'id': 'id_group', 'name': 'equipe'})

# Leitura da pasta de origem

In [ ]:
caminho = 'H:/XXXXXXXX/11. XXXXXXXXXXXX/1. XXXXXXX/xxxxxxxxx/xxxxxxxxx/xxxxxxxxxx/Teste/'

pasta = pathlib.Path(caminho)

ano = []
mes = []
dia = []
cod_loja = []
loja = []
unidade = []
arquivo = []
i = 0

for obj in pasta.iterdir():
    nome = obj.name
    if 'png' in nome:
        lista = nome.split('-')
        n = len(lista)
        cod_loja.append(f'{lista[0]}-{lista[1]}')
        ano.append(lista[n-4])
        mes.append(lista[n-3])
        dia.append(lista[n-2])
        arquivo.append(nome)
        uni = ''
        j = 2
        while j < (n-4):
            uni = f'{uni} {lista[j]}'
            j = j+1
        unidade.append(uni)
            
        i = i+1


d = ({'cod_loja': cod_loja,
      'unidade': unidade,
      'ano': ano,
      'mes': mes,
      'dia': dia,
      'arquivo': arquivo}
)

df_paradas = pd.DataFrame(data=d)

df_paradas['cod_loja'] = df_paradas['cod_loja'].str.strip()
df_paradas['ano'] = df_paradas['ano'].str.strip()
df_paradas['unidade'] = df_paradas['unidade'].str.strip()
df_paradas['mes'] = df_paradas['mes'].str.strip()
df_paradas['dia'] = df_paradas['dia'].str.strip()
df_paradas['arquivo'] = df_paradas['arquivo'].str.strip()

df_paradas


# Upload de imagens em massa

In [ ]:
user = 'xxxxxxxxxxx@xxxxxxxxxxxx.com.br/token'
pwd = 'xxxxxxxxxxxxxxxxxxxxxxx'
auth = (user, pwd)
url = 'https://xxxxxxxxxxxxxxxx.zendesk.com/api/v2/uploads.json'

token = []

img_url = []

cont = 0

#cada arquivo que foi lido na pasta é enviado para o zendesk
#o retorno é uma url e um token
#que são armazenados na df_paradas na linha correspondente ao arquivo enviado

while cont < i:
    local_filename = caminho + arquivo[cont]
    attachment_filename = caminho + arquivo[cont]

    # upload file
    
    params = {'filename': attachment_filename}
    headers = {'Content-Type': 'image/png'}
    with open(local_filename, 'rb') as f:
        response1 = requests.post(url, params=params, data=f, headers=headers, auth=auth)
    info = response1.json()
    token.append(info['upload']['token'])
    img_url.append(info['upload']['attachments'][0]['content_url'])
    
    cont = cont + 1
    

df_paradas['img_url'] = img_url

df_paradas['token'] = token

df_paradas

In [ ]:
#aqui é criada uma df que agrupa os dias de evento por unidade
# e une com as infos do executivo e do grupo (equipe)

df_consolidada = (
    df_paradas.groupby('cod_loja').agg(
        count = ('arquivo', 'count'),
        unidade = ('unidade', 'first')
    )
)

df_consolidada = df_consolidada.reset_index('cod_loja')

df_consolidada = df_consolidada.merge(df_cadastro, on = 'cod_loja', how = 'left')

df_consolidada = df_consolidada.rename(columns = {'email_executivo' : 'email'})

df_consolidada = df_consolidada.merge(df_users, on = 'email', how = 'left')

df_consolidada = df_consolidada.rename(columns = {'id' : 'id_executivo'})

df_consolidada = df_consolidada.merge(df_groups, on = 'equipe', how = 'left')

df_consolidada = df_consolidada.merge(df_mail, on = 'cod_loja', how = 'left')



df_consolidada

# Criação de lista de ids a partir da mailing list

In [ ]:
df_mail_aux = df_consolidada[['cod_loja', 'xmail']]

df_mail_aux['email'] = df_mail_aux['xmail'].str.split(';')

df_mail_aux = df_mail_aux.explode('email')

df_mail_aux = df_mail_aux.reset_index()

df_mail_aux['email'] = df_mail_aux['email'].str.strip()

df_mail_aux = df_mail_aux.loc[df_mail_aux['email'] != '']


df_mail_aux = df_mail_aux[['cod_loja', 'email']]

df_mail_aux = df_mail_aux.merge(df_users, on = 'email', how = 'left')

df_mail_aux = df_mail_aux.dropna()

df_mail_aux['id'] = df_mail_aux['id'].astype(np.int64)

df_mail_aux = df_mail_aux.groupby('cod_loja')['id'].apply(list)

df_mail_aux


In [ ]:
#quando uma unidade tem mais de um dia (mais de um arquivo/imagem) de evento
#as urls dos arquivos são unidas em uma string no formato html
#e os tokens reunidos em formato de lista

df_consolidada['body_img'] = ''

df_consolidada['uploads'] = ''

k = len(df_consolidada)

cont2 = 0

while cont2 < k:
    
    cont3 = 0
    
    body_img = ''
    uploads = []
    
    cod = df_consolidada['cod_loja'][cont2]
    
    df_paradas_aux = df_paradas.loc[df_paradas['cod_loja'] == cod]
    
    df_paradas_aux = df_paradas_aux.reset_index()
                                    
    qtd = df_consolidada['count'][cont2]
    
    while cont3 < qtd :
        url_aux = df_paradas_aux['img_url'][cont3]
        token_aux = df_paradas_aux['token'][cont3]
        
        body_img = body_img + f'<img src = {url_aux} width="500px"> <br>'
        uploads.append(token_aux)
        
        cont3 = cont3 +1
        
    df_consolidada['body_img'][cont2] = body_img
    
    df_consolidada['uploads'][cont2] = uploads
        
    cont2 = cont2 + 1
        
    
df_consolidada
    

# Criação dos tickets em massa

In [ ]:
body_inicio = (
    """<html lang = "pt-br">

    <head>
    <meta charset="utf-8">
    <title>E-mail Parada</title>
    </head>

    <body>
    
    <p>Prezados,</p>

    <p>Conforme gráficos a seguir, verificamos xxxxxxxxxxxxxxxxxxx.
    <br>Nestes casos, xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx.
    <br>Essa informação é essencial para xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx.</p>

    <p>Dessa forma, por gentileza, confirmar <b>xxxxxxxxxxx xxxxxxxxxxxxxxxxxxxxxx.</b></p>"""
)

body_fim =(
    """<p>Agradecemos desde já e qualquer dúvida, estamos à disposição.
    <br>
    <br>Atenciosamente,</p>


    </body>

    </html>"""    
    
)

In [ ]:
#aqui é criada uma lista de payloads com as informações contidas na df_consolidada

cont = 0

payload = []

while cont < k:
    
    html_body = body_inicio + df_consolidada['body_img'][cont] + body_fim
    
    
    uni = df_consolidada['unidade'][cont]
    
    tk = df_consolidada['uploads'][cont]
    
    sub = df_consolidada['id_exec'][cont].item()
    
    grp = df_consolidada['id_group'][cont].item()
    
    
    
    subject = (f'TESTE TICKET EM MASSA - {uni}')
    
    data = {'ticket': {'subject': subject,
                        'comment': {'html_body': html_body,
                                    'uploads': tk},
                        'submitter_id': sub,
                        'assignee_id': sub}}
    
    payload.append(json.dumps(data))
    
    cont = cont+1
    
payload

# Upload dos tickets em massa

In [ ]:
check = input("Deseja continuar e fazer upload dos tickets? Usuários internos e externos podem ser impactados.")

if check in ['Sim', 'sim', 'S', 's']:
    
    # Set the request parameters
    url = 'https://xxxxxxxxxxxxx.zendesk.com/api/v2/tickets.json'
    headers = {'content-type': 'application/json'}
    cont = 0

    while cont < k:
        # Do the HTTP post request
        response = requests.post(url, data=payload[cont], auth=(user, pwd),verify = False, headers=headers)

        # Check for HTTP codes other than 201 (Created)
        if response.status_code != 201:
            print('Status:', response.status_code, 'Problem with the request. Exiting.')
            exit()
        else:
            # Report success
            print(f'{cont} - Successfully created the ticket.')   

        cont = cont + 1
        
else:
    print('Upload NÂO REALIZADO')